# 🌃 About Segment Anything

The Segment Anything Model (SAM) produces high quality object masks from input prompts such as points or boxes, and it can be used to generate masks for all objects in an image. It has been trained on a dataset of 11 million images and 1.1 billion masks, and has strong zero-shot performance on a variety of segmentation tasks.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git

In [ ]:
# Imports

import matplotlib.pyplot as plt
import cv2
import os
import torch
from PIL import Image
import requests
from io import BytesIO
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

In [ ]:
checkpoint='/kaggle/input/segment-anything-models/sam_vit_h_4b8939.pth'

In [ ]:
pip install pillow requests

In [ ]:
# Get Image and save as sample_image

url = 'https://static.toiimg.com/thumb/msid-31346158,width-748,height-499,resizemode=4,imgsize-114461/.jpg'  # Replace this with the URL of your image
response = requests.get(url)

try:
    image = Image.open(BytesIO(response.content))
    image.save('sample_image.png')
    print("Image saved as sample_image.png")
except UnidentifiedImageError:
    print("The image format could not be identified.")


In [ ]:
img = cv2.imread('sample_image.png', cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert BGR to RGB for visualization
plt.imshow(img)

In [ ]:
# Model Instance initiate

sam = sam_model_registry["default"](checkpoint=checkpoint)

# Check for GPU
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sam.to(device)

In [ ]:
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(img)

In [ ]:
def show_mask(mask, ax):
    color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [ ]:
for mask in masks:
    if mask['area']<10_000:
        show_mask(mask['segmentation'], plt.gca())